# Симплекс алгоритъм с три променливи

Нека да разширим примера с производството на бира, като добавим и нов продукт - безалкохолна бира и да добавим още един ресурс - вода.

$$
\begin{align*}  
& x_1: \text{ светла бира (л.)}\\
& x_2: \text{ тъмна бира (л.)} \\
& x_3: \text{ безалкохолна бира (л.)} \\
\end{align*}
$$

$$
\begin{align*}
\max & \quad z(x) = 3x_1 + 2x_2 + x_3\\
\text{s.t.} \quad & x_1 + 2x_2 + x_3 \leq 20 \text{ Хмел}\\
\quad & 2x_1 + x_2 + 3x_3 \leq 30 \text{ Малц}\\
\quad & x_1 + x_2 + x_3 \leq 25 \text{ Вода}\\
\quad & x_1, x_2, x_3 \geq 0\\
\end{align*}
$$

Стандартната форма на задачата е:

$$
\begin{align*}
\max & \quad z(x) = 3x_1 + 2x_2 + x_3\\
\text{s.t.} \quad & x_1 + 2x_2 + 1x_3 + s_1 = 20 \text{ Хмел}\\
\quad & 2x_1 + x_2 + 3x_3 + s_2 = 30 \text{ Малц}\\
\quad & x_1 + x_2 + x_3 + s_3 = 25 \text{ Вода}\\
\quad & x_1, x_2, x_3, s_1, s_2, s_3 \geq 0\\
\end{align*}
$$


In [1]:
# Gurobi 
%pip install gurobipy --quiet

# Import the Gurobi library
import gurobipy as gp
from gurobipy import GRB
import numpy as np


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Create a new model

m = gp.Model("Beer 2")
m.Params.LogToConsole = 0

# Create variables
x = m.addMVar(3, name="x")
c = np.array([3, 2, 1])
c_coeffs = np.array([
    [1, 2, 1],
    [2, 1, 3],
    [1, 1, 1],
])
b = np.array([20, 30, 25])

# Objective function
m.setObjective(c @ x, GRB.MAXIMIZE)

# Constraints
m.addConstr(c_coeffs @ x <= b, name="c0")

m.write("beer.lp")

with open("beer.lp") as f:
    print(f.read())

m.optimize()

Restricted license - for non-production use only - expires 2026-11-23


Set parameter LogToConsole to value 0


\ Model Beer 2
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  3 x[0] + 2 x[1] + x[2]
Subject To
 c0[0]: x[0] + 2 x[1] + x[2] <= 20
 c0[1]: 2 x[0] + x[1] + 3 x[2] <= 30
 c0[2]: x[0] + x[1] + x[2] <= 25
Bounds
End



In [3]:
A = m.getA()
# print(A.toarray())
A_full = np.concatenate((A.toarray(), -np.identity(A.shape[1])), axis=0)

b_full = np.concatenate([b, np.zeros(A.shape[1])], axis=0)

import polytope as pc

P = pc.Polytope(A_full, b_full)
vertices = pc.extreme(P)
# print(vertices)


`polytope` failed to import `cvxopt.glpk`.


will use `scipy.optimize.linprog`


In [4]:
import plotly.graph_objects as go

# Plot the feasible region in 3D
# The feasible region is defined by the vertices of the polytope

fig = go.Figure(data=[
    go.Scatter3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        mode='markers',
        marker=dict(size=5, color='blue', opacity=1),
    )
])

# Plot the constraint planes

x1_values = np.linspace(-2, 10, 100)
x2_values = np.linspace(-2, 10, 100)
x1_grid, x2_grid = np.meshgrid(x1_values, x2_values)

# for i in range(A.shape[0]):
#     Z = (b[i] - A[i, 0] * x1_grid - A[i, 1] * x2_grid) / A[i, 2]  
#     fig.add_trace(go.Surface(
#         x=x1_grid,
#         y=x2_grid,
#         z=Z,
#         opacity=0.2,
#         name=f'Constraint {i+1}',
#     ))

# Mesh the vertices to create a 3D surface

from scipy.spatial import ConvexHull
hull = ConvexHull(vertices)

for simplex in hull.simplices:
    fig.add_trace(go.Mesh3d(
        x=vertices[simplex, 0],
        y=vertices[simplex, 1],
        z=vertices[simplex, 2],
        color='lightblue',
        opacity=0.7,
    ))

fig.update_layout(
    title='Допустимо множество',
    scene=dict(
        xaxis_title='x1 (светла бира)',
        xaxis=dict(nticks=4, range=[-0.3, 20]),
        yaxis=dict(nticks=4, range=[-0.3, 20]),
        zaxis=dict(nticks=4, range=[-0.3, 20]),
        yaxis_title='x2 (тъмна бира)',
        zaxis_title='x3 (безалкохолна бира)',
    ),
    width=800,
    height=800,
)

fig.show()

<!-- | Iteration-0   |       | $C_j$   | 3     | 2     | 1     | 0     | 0     | 0     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $s_1$     | 0     | 20          | 1     | 2     | 1     | 1     | 0     | 0     | $\frac{20}{1} = 20$ |
| $s_2$     | 0     | 30          | 2     | 1     | 3     | 0     | 1     | 0     | $\frac{30}{2} = 15$ |
| $s_3$     | 0     | 25          | 1     | 1     | 1     | 0     | 0     | 1     | $\frac{25}{1} = 25$ |
| $Z = 0$ |       | $Z_j$   | 0     | 0     | 0     | 0     | 0     | 0     |       |
|               |       | $Z_j - C_j$ | -3    | -2    | -1    | 0     | 0     | 0     |       |

| Итерация-1   |       | $C_j$   | 3     | 2     | 1     | 0     | 0     | 0     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $s_1$     | 0     | 5          | 0     | 1.5     | -0.5     | 1     | -0.5     | 0     | 3.333 |
| $x_1$     | 3     | 15          | 1     | 0.5    | 1.5     | 0     | 0.5     | 0     | 30 |
| $s_3$     | 0     | 10          | 0     | 0.5     | -0.5     | 0     | -0.5     | 1     | 20 |
| $Z = 45$ |       | $Z_j$   | 3     | 1.5     | 4.5     | 0     | 1.5     | 0     |       |
|               |       | $Z_j - C_j$ | -0.5    | 3.5    | 0    | 1.5     | 0     | 0     |       |

| Итерация-2   |       | $C_j$   | 3     | 2     | 1     | 0     | 0     | 0     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $x_2$     | 2     | 3.3          | 0     | 1     | -0.33     | 0.66     | -0.33     | 0     | 3.333 |
| $x_1$     | 3     | 13.3          | 1     | 0    | 1.66     | -0.33     | 0.66     | 0     | 30 |
| $s_3$     | 0     | 8.3          | 0     | 0     | -0.33     | -0.33     | -0.33     | 1     | 20 |
| $Z = 46.66$ |       | $Z_j$   | 3     | 2     | 4.33     | 0.33     | 1.33     | 0     |       |
|               |       | $Z_j - C_j$ | 0    | 0    | 0    | 1.5     | 0     | 0     |       |

 -->



| Итерация-0   |       | $C_j$   |     |     |     |     |     |     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $s_1$     |     |          |     |     |     |     |     |     |  |
| $s_2$     |     |          |     |     |     |     |     |     |  |
| $s_3$     |     |          |     |     |     |     |     |     |  |
| $Z =$ |       | $Z_j$   |     |     |     |     |     |     |       |
|               |       | $Z_j - C_j$ |     |     |     |     |     |     |       |


Трансформация :

- (2) <- (2) / 2
- (1) <- (1) - (2) 
- (3) <- (3) - (2)

| Итерация-1   |       | $C_j$   |     |     |     |     |     |     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $s_1$     |     |          |     |     |      |     |      |     | |
| $x_1$     |     |          |     |    |     |     |     |     | |
| $s_3$     |     |          |     |     |      |     |      |     | |
| $Z =$ |       | $Z_j$   |     |     |     |     |     |     |       |
|               |       | $Z_j - C_j$ |     |    |    |     |     |     |       |

Трансформация :

- (1) <- (1) / 1.5
- (2) <- (2) - 0.5 (1) 
- (3) <- (3) - 0.5 (1)

| Итерация-2   |       | $C_j$   |     |     |     |     |     |     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $x_2$     |     |          |     |     |      |     |      |     | |
| $x_1$     |     |          |     |    |     |      |     |     | |
| $s_3$     |     |          |     |     |      |      |      |     | |
| $Z = $ |       | $Z_j$   |     |     |     |     |     |     |       |
|               |       | $Z_j - C_j$ |    |    |    |     |     |     |       |

